## Key performance indicators
Intended cells:
- Aggregated risk profiles leading to events*
- Activity monitoring operating characteristic curve (AMOC)


*See [here](https://pubmed.ncbi.nlm.nih.gov/33234161/#&gid=article-figures&pid=fig-4-uid-3) for example, or below:



![hypotension risk curve](https://www.ncbi.nlm.nih.gov/pmc/articles/instance/7687996/bin/13054_2020_3379_Fig4_HTML.jpg)

## Aggregated risk profiles leading to events
Note that in example given there is a control group - for LTAF data we instead of samples of time sufficiently distant to afib

In [ ]:
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
def aggregatedRiskProfiles(model, data):
    #separate data into time's preceding afib and times sufficiently distant
    priorToAF = data[data.inAF]
    distantToAF = data[~data.inAF]

    priorToAF['modelConfidence'] = model.apply(priorToAF)
    distantToAF['modelConfidence'] = model.apply(distantToAF)


    numIncrements = 200
    timeBeforeToConsider = dt.timedelta(hours=3)
    confidenceAverages = {
        'priorToAF': list(),
        'distantToAF': list(),
        'timePrior': list()
    }
    pointsWithinSpan = lambda df, begin, end: df[(begin <= df['tte']) & (df['tte'] < end)]
    for i in range(numIncrements):
        bucketSpan = timeBeforeToConsider / numIncrements
        begin, end = i*bucketSpan, (i+1)*bucketSpan
        priorToAFPoints = pointsWithinSpan(priorToAF, begin, end) 
        confidenceAverages['priorToAF'].append(
            priorToAFPoints.modelConfidence
            .to_numpy().mean()
            )
        distantToAFPoints = pointsWithinSpan(distantToAF, begin, end)
        confidenceAverages['distantToAF'].append(
            distantToAFPoints.modelConfidence
            .to_numpy().mean()
        )
        confidenceAverages['timePrior'].append((begin+end) / 2)
    plt.plot(confidenceAverages['timePrior'], confidenceAverages['priorToAF'], label='AF samples')
    plt.plot(confidenceAverages['timePrior'], confidenceAverages['distantToAF'], label='Stable samples')
    plt.legend()
    plt.show()

## FOR THE MANUSCRIPT
- cross validation
- ablation (removing MMD regularization, restitching of annotations)

(nice to haves)
- neural architecture search [see here](https://github.com/kirthevasank/nasbot)
    - (get sample to run, then get it to run on tiny sample, then get it to run on whole dataset)
    - the only read additions are modifying the data loss

notes
- Kyle added afib internal points